* generatorは、暗黙にイテレータを作成してくれる
---
* イテレータ実装
  - __iter__(self) 実装（自分を返せば良い）
  - __next__(self) 実装
    + 途中までは、返すべきものを返す
    + 終了時には、raise StopIteration

In [1]:
class MyRange:
    def __init__(self, max):
        self.max = max
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        self.count += 1
        if self.count > self.max:
            raise StopIteration
        return self.count

In [2]:
for i in MyRange(4):
    print(i, end=', ')

1, 2, 3, 4, 

In [3]:
[*MyRange(7)]

[1, 2, 3, 4, 5, 6, 7]

* 普通のrange()と同じように使える
---
* generatorを使ったイテレータ実装
  - 関数実装（break & yield するだけで良い）

In [4]:
def myRange(max):
    count = 0
    while True:
        count += 1
        if count > max:
            break
        yield count

In [5]:
for i in myRange(10):
    print(i, end=', ')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 

---
* generator式は、内包表記もどきな構文で、yield文も書かない

In [6]:
for i in (x*x for x in range(10) if x % 2 == 0):
    print(i, end=', ')

0, 4, 16, 36, 64, 

---
* 無限に継続するgenerator

In [7]:
def getIntSequence():
    count = 0
    max = 0
    while True:
        count += 1
        if max > 0 and count > max:
            break
        y = yield count
        print('[{}]'.format(y), end='')
        if type(y) == type(max):
            max = y

* 普通に使うといつまでも終了しないので、呼び出し側は以下のように、どこかでmax指定を送る

In [8]:
gs = getIntSequence()
for i in gs:
    print(i, end=', ')
    if i == 4:
        i = gs.send(6)
        print(i, end='| ')

1, [None]2, [None]3, [None]4, [6]5| [None]6, [None]

* max指定を送っているsend()の使い方には注意が必要！
  - send()呼び出しで、generator側に制御が行き、再びyield文になるまでgenerator側実行になる
  - そして、次のyield戻り値をsend()が返してくる
  - 注意！ 直ぐ止めるとシーケンスが崩れ、StopIteration例外が発生する
---
* 直ぐ停止させたい場合

In [9]:
def getFinSequence():
    count = 0
    while True:
        count += 1
        y = yield count
        print('[{}]'.format(y), end='')
        if y == 'finish':
            break

* send()呼び出しをtry-exceptしてStopIteration例外を処理する

In [10]:
gs = getFinSequence()
for i in gs:
    print(i, end=', ')
    if i == 4:
        try:
            gs.send('finish')
        except(StopIteration):
            print('finished')

1, [None]2, [None]3, [None]4, [finish]finished
